In [ ]:
import json
import numpy as np
import pandas as pd

# Football DB

In [ ]:
# src/football_db_to_csv.py
fbdb_stats_old = pd.read_csv('../../data/football_db_player_stats_11_22.csv')
fbdb_stats = pd.read_csv('../../data/football_db_player_stats_12_05.csv')

fbdb_stats_old['filter_date'] = pd.to_datetime(fbdb_stats_old['date'])
fbdb_stats['filter_date'] = pd.to_datetime(fbdb_stats['date'])

# Append new records to old records
merge_df = pd.merge(fbdb_stats, fbdb_stats_old[['url', 'filter_date']], how='outer', on=['url', 'filter_date'], indicator=True)
new_stats = merge_df[lambda df: df['_merge'] == 'left_only']
fbdb_stats_new = pd.concat((fbdb_stats_old, new_stats), sort=False)
fbdb_stats_new.drop(columns=['filter_date', '_merge'], inplace=True)

fbdb_stats_new.to_csv('../../data/football_db_player_stats.csv', index=False)

# Player List
**NOT CURRENTLY USED**
- Positions: QB, RB (and variants), WR (and variants), TE
- Keep unique ids because players can be labelled under multiple variants

In [ ]:
with open('../../data/player_links.json', 'r') as f:
    players = []
    for line in f:
        players.append(json.loads(line))
        
players = pd.DataFrame(players)

In [ ]:
positions_tracked =  ['QB', 'RB', 'GLB', '3RB', 'FB', 'WR1', 'WR2', 'WR3', 'TE']
players = players.loc[lambda df: df.player_position.isin(positions_tracked), :]
players['player_position'].replace(['GLB', '3RB', 'FB', 'WR1', 'WR2', 'WR3'], ['RB', 'RB', 'RB', 'WR', 'WR', 'WR'], inplace=True)
players['player_id'] = players['player_link'].str.extract('([\d]+)', expand=False)
players.drop_duplicates('player_id', inplace=True)

In [ ]:
print(players.shape)
players.head()

In [ ]:
players.to_csv('../../data/player_list.csv', index=False)

# Player Stats
**NOT CURRENTLY USED. See football_db_player_stats.csv**

In [ ]:
with open('../../data/player_stats.json', 'r') as f:
    weekly_stats = []
    for line in f:
        weekly_stats.append(json.loads(line))
        
weekly_stats = pd.DataFrame(weekly_stats)

In [ ]:
weekly_stats['player_id'] = weekly_stats['url'].str.extract('([\d]+)', expand=False)
weekly_stats['away_game'] = weekly_stats['opp'].str.contains('@')
weekly_stats['opp'] = weekly_stats['opp'].str.replace('@', '')
weekly_stats['date'] = pd.to_datetime(weekly_stats['date'] + ' 2018', format='%b %d %Y')
weekly_stats = weekly_stats[weekly_stats['player_id'].isin(players['player_id'])]

In [ ]:
cols_in_order = ['player_id', 'player', 'date', 'week', 'opp', 'away_game', 
                 'pass_attempts', 'pass_completions', 'pass_percent',  'pass_yards', 'pass_ya', 'pass_td', 'pass_int',
                 'rush_attempts', 'rush_yards', 'rush_avg', 'rush_td', 
                 'reception', 'rec_yards', 'rec_avg', 'rec_td', 
                 'fumb_lost', 'ko_ret_td', 'ko_ret_yards', 'punt_ret_td', 'punt_ret_yards']

if all([True if col in weekly_stats.columns else False for col in cols_in_order]):
    weekly_stats = weekly_stats[cols_in_order]

In [ ]:
for col in weekly_stats.loc['pass_attempts' : 'punt_ret_yards']:
    weekly_stats[col] = weekly_stats[col].fillna(0)

In [ ]:
print(weekly_stats.shape)
weekly_stats.head()

In [ ]:
weekly_stats.to_csv('../../data/player_stats.csv', index=False)

In [ ]:
player_game_count = players.copy()
stats_group = weekly_stats.groupby('player_id').size()
stats_group = stats_group.reset_index()
stats_group.rename(columns={0: 'game_count'}, inplace=True)
player_game_count = player_game_count.merge(stats_group, how='left', on='player_id')
player_game_count['game_count'].fillna(0, inplace=True)

# Histogram of game stats per player
player_game_count['game_count'].value_counts().sort_index()

# Player News

- Need to filter news updates down to game updates
- Some players might not have game news updates

In [ ]:
with open('../../data/player_news_12_05.json', 'r') as f:
    news = []
    for line in f:
        news.append(json.loads(line))
        
news = pd.DataFrame(news)
news.set_index(news.index[::-1], inplace=True)
news.sort_index(inplace=True)

In [ ]:
news['player_id'] = news['url'].str.extract('([\d]+)', expand=False)
# Different entries have different time formats. One format is missing year
news['date2'] = pd.to_datetime(news['date'], errors='coerce', format='%a, %b %d, %Y %I:%M:%S %p')
news['date3'] = pd.to_datetime(news['date'], errors='coerce', format='%b %d - %I:%M %p')

news = news[news['player_id'].isin(players['player_id'])]

In [ ]:
def fill_dates(group):
    new_dates = []
    prev_year = None
    prev_month = None
    
    for row in group.itertuples():
        if row.date2 is not pd.NaT:
            prev_year = row.date2.year
            prev_month = row.date2.month
            new_dates.append(pd.NaT)
        elif row.date3 is not pd.NaT and prev_year is not None and prev_month is not None:
            if row.date3.month >= prev_month:
                new_dates.append(pd.to_datetime(str(prev_year) + str(row.date3)[4:]))
            else:
                new_dates.append(pd.to_datetime(str(prev_year + 1) + str(row.date3)[4:]))
        else:
            new_dates.append(row.date3)
    
    future_year = None
    future_moth = None
    new_dates = new_dates[::-1]
    for i, row in enumerate(group[::-1].itertuples()):
        if row.date2 is not pd.NaT:
            future_year = row.date2.year
            future_month = row.date2.month
        elif row.date3 is not pd.NaT and future_year is not None and future_month is not None and new_dates[i] is pd.NaT:
            if row.date3.month <= future_month:
                new_dates[i] = pd.to_datetime(str(future_year) + str(row.date3)[4:])
            else:
                new_dates[i] = pd.to_datetime(str(future_year - 1) + str(row.date3)[4:])
                
    return pd.DataFrame(new_dates[::-1], index=group.index, columns=['new_dates'])

In [ ]:
# This code assumes chronological order by player. Use this to populate missing year values
grouped_news = news.groupby('player_id')
output = grouped_news.apply(fill_dates)

news['new_dates'] = output['new_dates']
# For players with news entries only in the no year format. Assume year = 2018
news['new_dates'] = np.where(np.logical_and(news['new_dates'].notna(), news['new_dates'].dt.year == 1900),
                             pd.to_datetime('2018' + news['new_dates'].dt.strftime('%Y-%m-%d %H:%M:%S').str[4:]), news['new_dates'])

# Combine final date column and drop temp columns
news['date'] = np.where(news['date2'].notna(), news['date2'], news['new_dates'])
news.drop(columns=['date2', 'date3', 'new_dates'], inplace=True)

# Filter news before the 2018-2019 season
news = news[news['date'] >= pd.to_datetime('2018-09-06')]

# Filter news on game day
#news['game_date'] = news['date'].dt.date
#weekly_stats['game_date'] = weekly_stats['date'].dt.date
#news = pd.merge(news, weekly_stats[['player_id', 'game_date']], how='inner', on=['player_id', 'game_date'])
#news.drop(columns='game_date', inplace=True)

In [ ]:
news_old = news

In [ ]:
# Append new records to old records
merge_df = pd.merge(news, news_old[['url', 'date']], how='outer', on=['url', 'date'], indicator=True)
new_news = merge_df[lambda df: df['_merge'] == 'left_only']
news_new = pd.concat((news_old, new_news), sort=False)
news_new.drop(columns=['_merge'], inplace=True)

In [ ]:
print(news_new.shape)
news_new.head()

In [ ]:
news.to_csv('../../data/player_news_12_05.csv', index=False)

In [ ]:
player_news_count = players.copy()
news_group = news.groupby('player_id').size()
news_group = news_group.reset_index()
news_group.rename(columns={0: 'news_count'}, inplace=True)
player_news_count = player_news_count.merge(news_group, how='left', on='player_id')
player_news_count['news_count'].fillna(0, inplace=True)

# Histogram of game summary news updates per player
player_news_count['news_count'].value_counts().sort_index()